# Getting started with Data Quibbler

Below is a quick getting-started guide for *Data Quibbler*. 

You may also consult with the [[Examples]] to quickly demo some of the key *Data Quibbler* functionalities.

## Setting up

### Install

To install *Data Quibbler* use: [Maor]

`pip install pyquibbler`

### Import

`pyquibbler` is conventionally imported as `qb`:

In [1]:
import pyquibbler as qb

Some key required functions are `iquib` and `q`, which we import directly: 

In [2]:
from pyquibbler import iquib, override_all, q

As *Data Quibbler* needs to override normal *NumPy* and *matplotlib* functionalities, it needs to be imported and `override_all()` first:

In [3]:
qb.override_all()

Imports of numpy and matplotlib, if needed, should follow after we have run `override_all()`: 

In [4]:
import matplotlib.pyplot as plt
import numpy as np

### Graphics backend

*Data Quibbler* works well with `tk` and `widget` and [Maor].

If you are using *Jupiter Notebook*, use for example:

In [5]:
%matplotlib tk

## The quib object

A quib is an object that represents an output *value* as well as the *function* and *arguments* used to calculate this value.  There are two major types of quibs: *input-quibs* (i-quibs) which take any *Python* object as their argument and present it as their value (their function is trivially the identity function), and *function-quibs* (f-quibs) that produce their output value by applying any arbitrary function to any list of arguments including i-quibs, other f-quibs and any additional Python objects. 

### Input quibs

Any *Python* object can be transformed into an input-quib using the `iquib` command. For example:

In [6]:
my_lucky_number = iquib(7)
my_lucky_number

my_lucky_number = iquib(7)

Note that the repr of a quib shows its *name* (in this case `'my_lucky_number'`) and its *function* (in this case, `iquib(7)`). 

#### Getting the quib's value using `get_value()`
To get the output *value* of the quib, we use the `get_value()` method:

In [7]:
my_lucky_number.get_value()

7

#### Input quibs can represent objects of any standard class
Quibs can represent any *Python* object including Numeric, String, List, Tuple, Set, and
Dictionary. They can also represent NumPy ndarrays, matplotlib Artists as well as essentially objects of any arbitrary class.

For example:

In [8]:
city_data = iquib({'City':'Haifa', 'Population':279247})
city_data

city_data = iquib({'City': 'Haifa', 'Population': 279247})

In [9]:
hello_world = iquib(['Hello','World'])
hello_world.get_value()

['Hello', 'World']

#### Assigning new values to input quibs

Input quibs can be modified by assignments using standard *Python* assignment syntax:

In [10]:
hello_world[0] = 'Hi'
hello_world.get_value()

['Hi', 'World']

To completely replace the value of a quib, even with objects of a different type, use the `assign_value` method:

In [11]:
city_data.assign_value('anything')
city_data.get_value()

'anything'

### Functional quibs

#### Applying functions or operators to quibs creates a *function-quib* that performs these operations
Applying *Python* functions or operators to quibs, or to combinations of quibs with other objects, creates a *function-quib* whose function is to perform the indicated operation. Quibs can work with many standard *Python*, *NumPy* and *matplotlib* functions. Operators such as `+`, `-`, `/`, `*`, `**`, are also supported. Furthermore, indexing a quib also creates a function-quib. We can therefore easily define functional quibs using standard *Python* syntax.

As a simple example, let's start with an input quib `z` representing a numeric *NumPy* array:

In [12]:
z = iquib(np.array([3, 1, 2]))

We can use this quib in standard functions and operations, just like we would use a normal *NumPy* array. For example:

In [13]:
z_sqr = z ** 2
z_sqr

z_sqr = z ^ 2

The operation above created `z_sqr` which is a functional quib whose _function_ is to square the *value* of `z`.

We can similarly continue with additional downstream operations. Say, calculating the sum of all the elements of `z_sqr`:

In [14]:
sum_z_sqr = np.sum(z_sqr)
sum_z_sqr

sum_z_sqr = sum(z_sqr)

#### Quibs are typically defined declaratively (lazy evaluation) 

In general, quib operations are declarative; they define quibs with assigned functions, but do not execute these functions. For example, the statement above, `sum_z_sqr = np.sum(z_sqr)` creates a new quib whose function is to perform `np.sum` on the value of `z_sqr`, but this summation is not yet computed (deferred evaluation). The summation is only performed when we ask for the value of the quib.

#### Quib values are only calculated when their value is requested
To calculate the value of a funational-quib, we can use the `get_value()` method:

In [15]:
sum_z_sqr.get_value() # 3^2 + 1^2 + 2^2

14

The statement above cause the evaluation of `sum_z_sqr` (performing the function `np.sum` on the value of `z_sqr`). This operation, in turn, also caused the evaluation of `z_sqr`. 

#### Function quibs cache their calculated value
Following calculation of its value, a quib can cache the result to avoid unnecessary future calculations. Repeated requests for the value of the quib thereby do not require recalculations. 

### Upstream changes automatically propagate to affect downstream results.

When we make changes to a quib, these changes are automatically propagated downstream to affect the values of dependent quibs (recursively). For example, suppose we change one of the elements of our input quib `z`:

In [16]:
z[2] = 0

When such change is made, downstream dependent quibs are notified that their cached output is no longer valid (though, no calculation is being performed). Then, when we ask for the value of a downstream quib, it will get recalculated to reflect the upstream change:

In [17]:
sum_z_sqr.get_value() # 3^2 + 1^2 + 0^2

10

### Defining function quibs for a user-function or any unsupported functions

Many *Python*, *NumPy* and *matplotlib* functions are supported to work directly on quibs. Yet, some functions are not currently supported to work directly. Similarly, any user function that expect normal, non-quib, arguments cannot work directly with quib arguments. 
Therefore, Quibbler also provides a generic way to create functional quibs that apply any Python function (whether supported or not) or any user-functions to the value of quib arguments. This is done using the `q` function: the syntax `w = q(fun, *args, **kwards)` defines a new function-quib `w` whose function is to apply `fun` on the provided args and kwards arguments, while replacing any quib arguments with with respective values. 

For example, consider the Python function `str`. Applying `str(quib)` returns the string representation of the quib, rather than a function quib that performs `str` on the value of the quib: 

In [18]:
str(sum_z_sqr)

'sum_z_sqr = sum(z_sqr)'

Alternatively, to create a new function quib that performs the function `str` on the value of `sum_z_sqr`, we can use the `q` syntax:

In [19]:
str_sum_z_sqr = q(str, sum_z_sqr)
str_sum_z_sqr # this is a function-quib:

str_sum_z_sqr = str(sum_z_sqr)

In [20]:
str_sum_z_sqr.get_value() # the value of the function-quib is the string '10'

'10'

### Quib indexing is also interpreted functionally

When we index a quib, we get a new quib whose function is to perform the indexing operation. 

For example, we can define a function quib that calculates the middle value of consecutive elements of an array:

In [21]:
r = iquib(np.array([0., 3., 2., 5., 8.]))
r_middle = (r[0:-1] + r[1:]) / 2
r_middle

r_middle = (r[0:-1] + r[1:]) / 2

In [22]:
r_middle.get_value()

array([1.5, 2.5, 3.5, 6.5])

Note that `r_middle` is defined functionaly; if its argument change it will get re-evaluated:

In [23]:
r[-1] = 13.
r_middle.get_value()

array([1.5, 2.5, 3.5, 9. ])

### Quibs can be combined with graphics functions, readily forming interactive GUIs.

Graphical functions too can be applied to quibs. Such graphics is bi-directionally linked to the quib: changes to upstream quibs refresh the graphics, and user interactions with the graphics can change upstream quibs (see also a separate chapter on [[Inverse assignments]]).

Consider, for example, the presentation of a graphics marker whose x-y coordinates are specified by quibs. Moving the graphics marker will change the values of x and y. Any quib graphics that depend on x or y will immediately refresh. 

In [24]:
# define and plot a curve:
curve_function = lambda v: 4 * v ** 2 - v ** 3
graph_xs = np.arange(0, 4, .2)
graph_ys = curve_function(graph_xs)
plt.figure(figsize=(4,3))
plt.plot(graph_xs, graph_ys, 'k')
plt.axis([0, 4, 0, 12])

# define x-y quibs:
point_x = iquib(3.)
point_y = q(curve_function, point_x)

# Plot the point, use picker=True to allow dragging
plt.plot(point_x, point_y, marker='o', markerfacecolor='c', 
         markersize=18, picker=True, pickradius=20)

# Define and plot text (this text will change when the marker is dragged):
xy_str = q("X={:.2f}, Y={:.2f}".format, point_x, point_y)
plt.text(point_x, point_y + .6, xy_str, 
         horizontalalignment="center",
         verticalalignment="bottom", fontsize=13);

[[/images/demo_gif/quibdemo_drag_on_curve.gif]]